# Imports

You will need to import MNE in order to use its functions to load and preprocess your data. Numpy is an extremely useful package used for mathematical and matrix operations. Scipy is a package for scientific computing and contains a wide range of tools for data analysis and running statistical tests. Pandas is the most convenient way of reading event codes from CSV files, but can also be used for analyzing tables of data. Matplotlib is Python's primary plotting library, and will be useful for visualizing results. The fdrcorrection() function can be imported from the statsmodels package for the purposes of adjusting p-values for multiple comparisons using the Benjamini/Hochberg method (useful for EEG analyses where you might make hundreds of comparisons). The line "%matplotlib qt" ensures that any time you try to plot your data, it will appear in a separate pop-up window and not just embed itself in the Jupyter notebook.

Remember that if you wish to run artifact blocking, you will also need to import the run_ab function from artifact_blocking.py.

In [ ]:
%matplotlib qt
import mne
import numpy as np
import pandas as pd
import scipy.stats as ss
import matplotlib.pyplot as plt
from statsmodels.stats.multitest import fdrcorrection

# Example Preprocessing

The code below just performs basic data loading, referencing, filtering, and epoching. It is designed to allow you to quickly load some data into this notebook in case you would like to test out any of the analysis functions below.

In [ ]:
# Load data and event codes
# Set the path to the current session's EEG recording
filepath = 'D://Documents/EEG_Workshop/data/Adult_08_T.mff'

# Load the EEG file and extract trigger times
eeg = mne.io.read_raw_egi(filepath, preload=True)
eeg.rename_channels({'E129': 'Cz'})
montage = mne.channels.make_standard_montage('GSN-HydroCel-129')
eeg = eeg.set_montage(montage)
events = mne.find_events(eeg)

# Select only events with a specific trigger code (e.g., 1)
# Only necessary if the number of trigger pulses in your recording does not match the number of events listed in your event code files
events = events[events[:, 2] == 1]
# Set the list of event code files for this session
evcode_files = ['erica/Video_Trigger_files/DupWav6min5.csv',
                'erica/Video_Trigger_files/DupWav6min7.csv',
                'erica/Video_Trigger_files/DupWav6min9.csv']
# Re-code events
evcodes = []
for f in evcode_files:
    evcodes.append(pd.read_csv(f, header=None))
evcodes = np.concatenate(evcodes).flatten()
events[:, 2] = evcodes

# Drop trigger channels once events have been extracted
eeg = eeg.pick_types(eeg=True)

# Re-reference to common average
eeg = eeg.set_eeg_reference(ref_channels='average')

# Pick desired channels
eeg = eeg.pick_channels(['E11', 'E47', 'E98'])

# .5 Hz - 40 Hz bandpass filter
l_freq = .5
h_freq = 40
l_trans_bandwidth = 'auto'
h_trans_bandwidth = 'auto'
filter_length = 'auto'
fir_window = 'hamming'
phase = 'zero'
picks = None
eeg = eeg.filter(l_freq, h_freq, method='fir', fir_design='firwin', l_trans_bandwidth=l_trans_bandwidth, h_trans_bandwidth=h_trans_bandwidth, filter_length=filter_length, fir_window=fir_window, phase=phase, pad='reflect_limited', picks=picks)

# Epoch data after defining ROIs, baseline correct each epoch by its own average
tmin = 1.
tmax = 29.
baseline = (None, None)
eeg = mne.Epochs(eeg, events, tmin=tmin, tmax=tmax, baseline=baseline, preload=True)

# Event-Related Potentials (MNE)

You can calculate ERPs by calling the average() method on epoched data. ERPs are stored in a channel x time array inside of an MNE Evoked object. Each row of the data contains the ERP for a single channel, averaged over all events. 

If your EEG object contains epochs from multiple conditions, average() will combine all conditions into one overall ERP:

In [ ]:
erp = eeg.average()

In order to get separate ERPs for each condition, you will need to index your EEG object with the name of each condition and run average() for each condition. The code below produces a list containing the Evoked objects for each condition you specify in condition_names:

In [ ]:
erp1 = eeg['1'].average()
erp2 = eeg['2'].average()

## Basic ERP Visualization

### ERP Butterfly Plots

In [ ]:
fig1 = erp1.plot()
fig2 = erp2.plot()

### Plot Topomaps of ERPs

Note that if you analyzing ROIs rather than channels, it will not be possible to plot a topomap.

In [ ]:
fig1 = erp1.plot_topomap()
fig2 = erp2.plot_topomap()

### Plot Comparison of Conditions

Click on any channel location to view a plot of that channel's ERP in each condition. Note that you will not be able to plot a topomap if you are analyzing ROIs instead of channels.

In [ ]:
fig = mne.viz.plot_evoked_topo([erp1, erp2])

# Event-Related Potentials (Manual)

While MNE is the most convenient way to calculate ERPs, it only gives you the mean of each ERP. If you want to plot your ERPs with confidence intervals or shaded error regions, you will need to calculate them yourself outside of MNE. Here is some example code for calculating the mean and standard error across events and plotting a comparison of two conditions for a given channel.

In [ ]:
# Calculate means and standard errors for condition 1
m1 = eeg['1'].get_data().mean(axis=0) * 1000000
s1 = eeg['1'].get_data().std(axis=0) * 1000000 / np.sqrt(eeg['1']._data.shape[0])

# Calculate means and standard errors for condition 2
m2 = eeg['2'].get_data().mean(axis=0) * 1000000
s2 = eeg['2'].get_data().std(axis=0) * 1000000 / np.sqrt(eeg['2']._data.shape[0])

In [ ]:
ch_name = 'E98'
condition_names = ['1', '2']
color1 = 'C0'
color2 = 'C3'
line_opacity = 1
error_opacity = .2

# Identify index of desired channel
ch = mne.pick_channels(eeg.ch_names, [ch_name])[0]

# Create axis lines at 0 uV and time 0
plt.axvline(0, c='k', ls='--')
plt.axhline(0, c='k', ls='--')

# Plot condition 1
l1, = plt.plot(eeg.times, m1[ch, :], c=color1, alpha=line_opacity)
plt.fill_between(eeg.times, m1[ch, :] - s1[ch, :], m1[ch, :] + s1[ch, :], color=color1, alpha=error_opacity)

# Plot condition 2
l2, = plt.plot(eeg.times, m2[ch, :], c=color2, alpha=line_opacity)
plt.fill_between(eeg.times, m2[ch, :] - s2[ch, :], m2[ch, :] + s2[ch, :], color=color2, alpha=error_opacity)

# Add labels and legend
plt.title(ch_name)
plt.xlabel('Time (s)')
plt.ylabel('Amplitude (uV)')
plt.legend([l1, l2], condition_names)

plt.show()

# Time-Frequency Decomposition

## Morlet Wavelet Decomposition

### Settings:

The first three settings affect the decomposition process:

- **freqs**: An array containing the frequencies you wish to decompose. If you wish to use linearly-spaced or log-spaced frequencies, you can use one of the numpy functions provided to generate an array of frequencies for you.
- **n_cycles**: The number of wavelet cycles to use for each frequency. Can be an array with one value for each frequency, or a single value to use for all frequencies.
- **picks**: Set to None to run on all channels. Set to a list of channel indices to select only those channels.

The final three settings affect what information is returned:

- **decim**: Sets decimation level to apply after decomposition, in order to reduce memory usage. For instance, if decimation level is set to 2, every other time point will be returned. If decimation level is set to 3, every third time point will be returned. If decimation level is 1 (default), all time points will be returned.
- **average**: If True, returns time-frequency information averaged across all epochs. If False, returns time-frequency information for each epochs.
- **output**: Set to "power" to only output power. Set to "complex" to return power and phase information. Note that average must be set to False to use "complex".

In [ ]:
# freqs = np.linspace(3, 40, num=15)  # Create linearly-spaced frequencies
freqs = np.logspace(np.log10(3), np.log10(40), num=15)  # Create log-spaced frequencies
n_cycles = 5
picks = None
decim = 1
average = True
output = 'power'
power, itc = mne.time_frequency.tfr_morlet(eeg, freqs, n_cycles, decim=decim, picks=picks, average=average, output=output)

## Multitaper Method

The settings for the multitaper method of time-frequency decomposition are identical to those for Morlet wavelet decomposition, with the exception that the function cannot be set to output complex numbers.

In [ ]:
# freqs = np.linspace(3, 40, num=15)  # Create linearly-spaced frequencies
freqs = np.logspace(np.log10(3), np.log10(40), num=15)  # Create log-spaced frequencies
n_cycles = 5
picks = None
decim = 1
average = True
power, itc = mne.time_frequency.tfr_multitaper(eeg, freqs, n_cycles, decim=decim, picks=picks, average=average)

# Frequency Tagging

The code below implements the frequency-tagging procedure from Nozaradan, Peretz, Missal, and Mouraux (2011). The steps involve 1) running the Fast Fourier Transform on your average data, 2) subtracting from each frequency's amplitude the average amplitude of its neighboring frequencies, and 3) replacing each frequency's amplitude with the average amplitude of itself and its neighbors.

This process requires the following settings and provides you with the following outputs:
### Settings:
- **data**: A 2D channels x time array containing the average time series for each channel. Note that if your averaged data is stored as an mne.Evoked object, you will need to extract the data array from the object as shown.
- **nfreqs**: An integer indicating how many frequencies to decompose via FFT. Note that half of the frequencies produced by FFT will be negative. As we only care about nonnegative frequencies, the code will automatically drop the negative frequencies, leaving you with only half *nfreqs* frequencies.
- **sampling_rate**: A float indicating the number of samples recorded per second.

### Outputs:
- **freqs**: A 1D array of the frequencies decomposed by the FFT.
- **amp**: A 2D channels x frequencies array containing each channel's raw amplitude at each frequency. 
- **norm_amp**: A 2D channels x frequencies array containing each channel's amplitude at each frequency after normalization.

If you only care about a specific few frequencies of interest, you can use the code from the second cell below to return each channel's normalized amplitude at each of those target frequencies. In the event that the exact target frequencies were not among those decomposed by the FFT, the nearest possible frequency will be used instead (although you should try to set up your FFT so that the exact frequencies of interest are analyzed). This process requires you to specify one setting and provides you with the following two outputs:

### Settings:
- **target_freqs**: A list of the frequencies of interest.

### Outputs:
- **closest_freqs**: A 1D array of the closest frequencies to the target frequencies.
- **target_amp**: A 2D channels x frequencies array containing the each channel's amplitude at each target frequency.

In [ ]:
# FFT Settings
data = erp1.data
nfreqs = data.shape[1]
sampling_rate = erp1.info['sfreq']

# Run FFT
freqs = np.fft.fftfreq(nfreqs, d=1/sampling_rate)
amp = np.abs(np.fft.fft(data, n=nfreqs, axis=1))
amp = amp[:, freqs >= 0]
freqs = freqs[freqs >= 0]

# Subtract average of bins +/- 3-5
sub_amp = np.empty_like(amp)
reference_bins = np.array([-5, -4, -3, 3, 4, 5])
for i in range(sub_amp.shape[1]):
    reference_inds = reference_bins + i
    reference_inds = reference_inds[(reference_inds >= 0) & (reference_inds < amp.shape[1])]
    sub_amp[:, i] = amp[:, i] - np.mean(amp[:, reference_bins], axis=1)
sub_amp[sub_amp < 0] = 0  # Don't allow negative amplitudes

# Average amplitudes across neighboring bins
norm_amp = np.zeros((sub_amp.shape[0], len(freqs)))
bins_to_avg = np.array([-1, 0, 1])
for i in range(sub_amp.shape[1]):
    inds_to_avg = bins_to_avg + i
    inds_to_avg = inds_to_avg[(inds_to_avg >= 0) & (inds_to_avg < sub_amp.shape[1])]
    norm_amp[:, i] = np.mean(sub_amp[:, inds_to_avg], axis=1)
del sub_amp

In [ ]:
# Get amplitudes from list of target frequencies
target_freqs = [.8, 1.2, 2.4]

closest_freqs = np.empty(len(target_freqs))
target_amp = np.empty((amp.shape[0], len(target_freqs)))
for i, f in enumerate(target_freqs):
    ind = np.argmin(np.abs(freqs - f))
    closest_freqs[i] = freqs[ind]
    target_amp[i] = amp[:, ind]
    
print('Frequencies:\n', closest_freqs)
print('Amplitudes:\n', target_amp)

Below you'll find some basic example code for plotting power spectra before and after frequency tagging for three regions of interest:

In [ ]:
min_freq = 0
max_freq = 20

plt.subplot(321)
plt.title('Raw')
plt.ylabel(erp1.ch_names[0])
plt.plot(freqs, amp[0, :])
plt.xlim(min_freq, max_freq)
plt.subplot(322)
plt.title('Normalized')
plt.plot(freqs, norm_amp[0, :])
plt.xlim(min_freq, max_freq)

plt.subplot(323)
plt.ylabel(erp1.ch_names[1])
plt.plot(freqs, amp[1, :])
plt.xlim(min_freq, max_freq)
plt.subplot(324)
plt.plot(freqs, norm_amp[1, :])
plt.xlim(min_freq, max_freq)

plt.subplot(325)
plt.ylabel(erp1.ch_names[2])
plt.plot(freqs, amp[2, :])
plt.xlim(min_freq, max_freq)
plt.subplot(326)
plt.plot(freqs, norm_amp[2, :])
plt.xlim(min_freq, max_freq)

plt.gcf().set_size_inches((12, 10))
plt.gcf().tight_layout()

# Statistics

Python offers convenient functions for running most basic statistics (t-tests, correlations, etc.). While you can perform more advanced statistics in Python as well, it is recommended that you export your data into R or SPSS if you wish to run ANOVAs or anything more complex. 

### Simulated Data (for test-running stats functions)

In [ ]:
# Simulate random ERPs with 1000 time points from 30 subjects and 2 conditions (data should be organized with one subject per row)
cond1 = np.random.randn(30, 1000)
cond2 = np.random.randn(30, 1000)

# Simulate 100 data points for two measures
score1 = np.random.randn(100)
score2 = np.random.randn(100)

### Independent Samples *t*-Tests

In [ ]:
# Compare each time point between conditions, giving one t and p-value per time point
tvals, pvals = ss.ttest_ind(cond1, cond2)

# Identify time points that differ signficantly between conditions after Benjamini-Hochberg FDR correction
sig, pvals_adjusted = fdrcorrection(pvals, alpha=0.05)

### Dependent Samples *t*-Tests

In [ ]:
# Compare each time point between conditions, giving one t and p-value per time point
tvals, pvals = ss.ttest_rel(cond1, cond2)

# Identify time points that differ signficantly between conditions after Benjamini-Hochberg FDR correction
sig, pvals_adjusted = fdrcorrection(pvals, alpha=0.05)

### Pearson Correlation Coefficient

In [ ]:
# Calculate Pearson r between two sets of scores
rval, pval = ss.pearsonr(score1, score2)

# Exporting Data from Python

### To CSV

In [ ]:
# Example: You have 30 participants with 5 dependent measures and 2 conditions
data = np.random.randn(30, 5)
conditions = np.random.choice([1, 2], 30)
col_names = ['score1', 'score2', 'score3', 'score4', 'score5']

# Organize your data as a dataframe
df = pd.DataFrame(data, columns=col_names)

# Add column indicating which participants were in which condition
df['condition'] = conditions

# Save to CSV
df.to_csv('C://Users/jpazd/Downloads/example_data.csv', header=True, index=False)